#IMPORT LIBRARY AND DATA

In [1]:
# Pandas, Bokeh, Seaborn, Mathplot library
import pandas as pd
import numpy as np
import re
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import warnings
import math

# Datetime library
import datetime
from datetime import datetime
from datetime import timedelta

# Apriori library
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
# Crearte dataframe
url = 'https://raw.githubusercontent.com/thienbao8112/Global-Store/main/SuperStore.csv'
df = pd.read_csv(url)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# Get general information about the data
df.info()
print("-" * 50)
print(pd.value_counts(df.dtypes))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          51290 non-null  int64  
 1   Order ID        51290 non-null  object 
 2   Order Date      51290 non-null  object 
 3   Ship Date       51290 non-null  object 
 4   Ship Mode       51290 non-null  object 
 5   Customer ID     51290 non-null  object 
 6   Customer Name   51290 non-null  object 
 7   Segment         51290 non-null  object 
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Postal Code     9994 non-null   float64
 12  Market          51290 non-null  object 
 13  Region          51290 non-null  object 
 14  Product ID      51290 non-null  object 
 15  Category        51290 non-null  object 
 16  Sub-Category    51290 non-null  object 
 17  Product Name    51290 non-null 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-3-404fd99c8d45>:4: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print(pd.value_counts(df.dtypes))


#PREPROCESSING

In [4]:
# Check to see if there are any missing values and eliminate from data set
i = df.shape[0]
print("Initial shape of data:", df.shape)
df = df.dropna()
print("Shape of data after drop:", df.shape)
print("-"*50)
print("Missing_values_eliminated:",  i - df.shape[0])

Initial shape of data: (51290, 24)
Shape of data after drop: (9994, 24)
--------------------------------------------------
Missing_values_eliminated: 41296


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
df['Sub-Category'] = df['Sub-Category'].astype(str)
df['Order ID'] = df['Order ID'].astype(str)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-5-c9b8fd91406f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sub-Category'] = df['Sub-Category'].astype(str)
<ipython-input-5-c9b8fd91406f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [6]:
#Check to see if there are any duplicated values and eliminate from data set
i = df.shape[0]
print("Initial shape of the dataset : ", df.shape)
df = df.drop_duplicates(keep="first", ignore_index=False)
print("Shape of the dataset after dropping the duplicate rows : ", df.shape)
print("-"*50)
print("Duplicated_values_eliminated:",  i - df.shape[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Initial shape of the dataset :  (9994, 24)
Shape of the dataset after dropping the duplicate rows :  (9994, 24)
--------------------------------------------------
Duplicated_values_eliminated: 0


In [7]:
# Create column to keep
columns_to_keep = ['Order ID', 'Sub-Category','Quantity']

# Create dataframe with necessary column
df = df[columns_to_keep]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# Processing into transactional
basket = (df.groupby(['Order ID', 'Sub-Category'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Order ID'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# Show data
basket.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sub-Category,Accessories,Appliances,Art,Binders,Bookcases,Chairs,Copiers,Envelopes,Fasteners,Furnishings,Labels,Machines,Paper,Phones,Storage,Supplies,Tables
Order ID,,,,,,,,,,,,,,,,,
CA-2011-100006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
CA-2011-100090,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
CA-2011-100293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
CA-2011-100328,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2011-100363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0


#APRIORI

In [10]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

# Encoding the datasets
basket_encoded = basket.applymap(hot_encode)
basket= basket_encoded

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-10-5237cc0abb71>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded = basket.applymap(hot_encode)


In [11]:
# Applying Apriori
frq_items = apriori(basket, min_support=0.01, use_colnames=True)

# Show results in data frame
rules = association_rules(frq_items, metric="lift", min_threshold=1)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Binders),(Appliances),0.262727,0.090038,0.025953,0.098784,1.097140,0.002298,1.009705,0.120090
1,(Appliances),(Binders),0.090038,0.262727,0.025953,0.288248,1.097140,0.002298,1.035857,0.097300
2,(Furnishings),(Appliances),0.175085,0.090038,0.015971,0.091220,1.013129,0.000207,1.001301,0.015710
3,(Appliances),(Furnishings),0.090038,0.175085,0.015971,0.177384,1.013129,0.000207,1.002794,0.014241
4,(Appliances),(Paper),0.090038,0.237772,0.021761,0.241685,1.016458,0.000352,1.005160,0.017793
5,(Paper),(Appliances),0.237772,0.090038,0.021761,0.091520,1.016458,0.000352,1.001631,0.021242
6,(Appliances),(Phones),0.090038,0.162507,0.015173,0.168514,1.036964,0.000541,1.007224,0.039173
7,(Phones),(Appliances),0.162507,0.090038,0.015173,0.093366,1.036964,0.000541,1.003671,0.042563
8,(Phones),(Art),0.162507,0.145937,0.024755,0.152334,1.043833,0.001040,1.007546,0.050140
9,(Art),(Phones),0.145937,0.162507,0.024755,0.169631,1.043833,0.001040,1.008578,0.049168
